# Group Proposal

Group Member: Erin Da, Lesley Mai, Xiaofei Sun, Yuchen Xu (Group 46)

## Introduction

Dropout rates and academic success play vital roles within higher education institutions, the academic success not only affects individual students but also affects society at large. Therefore, harnessing the available data to predict students’ dropout and academic success status holds significant worth.

This dataset is created from a higher education institution related to students enrolled in different undergraduate degrees, such as agronomy, design, education, nursing, journalism, management, social service, and technologies. The dataset includes information known at the time of student enrollment (academic path, demographics, and social-economic factors) and the students' academic performance at the end of the first and second semesters. The data is used to build classification models to predict students' dropout and academic success. The problem is formulated as a three category classification task, in which there is a strong imbalance towards one of the classes.

This project highlights key factors that lead students to drop out, with the major cause being socioeconomic conditions.

Given a student with below related features, we can classify the student’s academic status (enrolled, graduate or dropout).

## Preliminary exploratory data analysis

## Methods

Explain how you will conduct either your data analysis and which variables/columns you will use. 

The columns we choose: 
`Course`, 
`Daytime/evening attendance`, 
`Mother's qualification`,
`Father's qualification`,
`Mother's occupation`,
`Father's occupation`,
`Admission grade`,
`Displaced`,
`Debtor`,
`Scholarship holder`,
`Curricular units 1st sem (enrolled)`,
`Curricular units 1st sem (grade)`,
`Curricular units 2nd sem (enrolled)`,
`Curricular units 2nd sem (grade)`

Describe at least one way that you will visualize the results.

## Citation: 
background: https://www.mdpi.com/2306-5729/7/11/146